<h3>Nomes dos integrantes</h3>
<ul>
    <li>José Paulo Marinho - 02201062</li>
    <li>Kalil Bego - 02201046</li>
    <li>Patrick Lessa Teixeira - 02201058</li>
    <li>Victor Alves Barbosa - 02201039</li>
</ul>

<h3>Tratamento do primeiro data set de Cachorros</h3>

In [1]:
import pandas as pd
dataset_dogs = pd.read_csv('../data/NYC_Dog_Licensing_Dataset.csv')

<b>Removendo colunas que não iremos utilizar</b>

In [ ]:
dataset_dogs.drop(['RowNumber', 'CensusTract2010', 'NTA', 'CityCouncilDistrict', 'CommunityDistrict', 'CongressionalDistrict', 'StateSenatorialDistrict', 'LicenseIssuedDate', 'LicenseExpiredDate'], axis=1, inplace=True)
dataset_dogs.drop(dataset_dogs.index[67146:122203], axis=0, inplace=True)

In [ ]:
from datetime import datetime

# substituindo os valores do campo AnimalBirthMonth para retirar a hora deixa somente a data
dataset_dogs['AnimalBirthMonth'] = pd.to_datetime(dataset_dogs['AnimalBirthMonth'], format="%d/%m/%Y %H:%M:%S %p")
dataset_dogs['AnimalBirthMonth'] = pd.to_datetime(dataset_dogs['AnimalBirthMonth'])
dataset_dogs['AnimalBirthMonth'] = dataset_dogs['AnimalBirthMonth'].map(lambda x: datetime.strftime(x,"%d/%m/%Y"), na_action='ignore')

<h3>Tratamento do segundo dataset_dogs de Gatos</h3>

In [ ]:
dataset_cats = pd.read_csv('../data/cats.csv')

#removendo colunas que não iremos utilizar
dataset_cats.drop(['Unnamed: 0', 'id', 'url', 'med_photos', 'size', 'age'], axis=1, inplace=True)

import ast
dataset_cats["photos"] = dataset_cats["photos"].apply(ast.literal_eval)
dataset_cats['photos'] = dataset_cats['photos'].map(lambda foto: foto[0]["large"])

dataset_cats = dataset_cats.rename(columns={'photos': 'link_photo'})

<b>Salvando dataset_dogss</b>

In [5]:
dataset_dogs.to_csv('../data/dogs_clean.csv')
dataset_cats.to_csv('../data/cats_clean.csv')

<h3>União dos datasets "limpados" acima.</h3>

In [ ]:
new_dataset_data_dogs = {
    "AnimalName": dataset_dogs["AnimalName"],
    "AnimalGender": dataset_dogs["AnimalGender"],
    "AnimalBirthMonth": dataset_dogs["AnimalBirthMonth"],
    "BreedName": dataset_dogs["BreedName"],
    "Borough": dataset_dogs["Borough"],
    "ZipCode": dataset_dogs["ZipCode"],
    "Type": "Dog",
    "Image": ""
}

new_dataset_data_cats = {
    "AnimalName": None,
    "AnimalGender": dataset_cats["gender"],
    "AnimalBirthMonth": None,
    "BreedName": dataset_cats["breed"],
    "Borough": None,
    "ZipCode": None,
    "Type": dataset_cats["type"],
    "Image": dataset_cats["link_photo"]
}

new_dataset_dogs = pd.DataFrame(new_dataset_data_dogs)
new_dataset_cats = pd.DataFrame(new_dataset_data_cats)

union_dataset = new_dataset_dogs.append(new_dataset_cats)

In [7]:
import csv

dicionario_ceps = {}

with open('../data/ceps.txt', 'r') as file:
    csvreader = csv.reader(file, delimiter=';')
    for linha in csvreader:
        dicionario_ceps[linha[0]] = linha

In [8]:
import numpy as np
np.random.seed(77)

chaves_dicionario = np.array(list(dicionario_ceps.keys()))

def get_cep():
    key = np.random.choice(chaves_dicionario)
    info_cep = dicionario_ceps[f'{key}']
    cep = info_cep[0]
    return cep

def get_cidade(cep):
    return dicionario_ceps[cep][1]

union_dataset["ZipCode"] = union_dataset["ZipCode"].map(lambda _: get_cep())
union_dataset["Borough"] = union_dataset["ZipCode"].map(lambda cep: get_cidade(cep))

union_dataset.to_csv("../data/dataset_union.csv")

<h3>Tratamento das fotos da base</h3>
<ul>
    <li>Obter a foto da URL</li>
    <li>Converter para base64</li>
</ul>

In [9]:
import requests
import base64

def get_base64_from_photo_url(url):
    if not url:
        return None
    
    response = requests.get(url)
    imagem_bytes = response.content
    imagem_base64 = base64.b64encode(imagem_bytes).decode('utf-8')
    return imagem_base64

In [ ]:
def is_base64(s):
    try:
        base64.b64decode(s)
        return True
    except Exception:
        return False

for index, row in union_dataset.iterrows():
    if row["Image"] and row["Type"] == "Cat" and not is_base64(row["Image"]):
        union_dataset.loc[index, 'Image'] = get_base64_from_photo_url(row["Image"])

union_dataset.to_csv("../data/dataset_union.csv")

<h3>Encontrar fotos dos cachorros em outro dataset</h3>

In [11]:
import os

dir_atual = os.getcwd()

dir_destino = os.path.abspath(os.path.join(dir_atual, os.pardir, 'train'))

os.chdir(dir_destino)

arquivos_jpg = [f for f in os.listdir() if f.endswith('.jpg')]

fotos = set(arquivos_jpg)

In [13]:
dataset_breed_dogs = pd.read_csv('../data/labels.csv')

In [ ]:
import base64
from PIL import Image

def find_image_from_name_dog(breed_name):
    if breed_name == 'Unknown' or not breed_name:
        return None, dataset_breed_dogs
    breeds_founded = dataset_breed_dogs[dataset_breed_dogs['breed'].apply(str.lower).str.contains(str(breed_name).lower(), case=False)]
    
    if breeds_founded.empty:
        return None, dataset_breed_dogs
    
    first_breed = breeds_founded.iloc[0]
    df = dataset_breed_dogs.drop(breeds_founded.index[0])
    info_breed = first_breed.to_dict()
    indice_arquivo = arquivos_jpg.index(f"{info_breed['id']}.jpg")
    nome_arquivo = arquivos_jpg[indice_arquivo]
    caminho_completo = os.path.join(dir_destino, nome_arquivo)
    img = Image.open(caminho_completo)
    img_bytes = img.tobytes()
    return base64.b64encode(img_bytes).decode('utf-8'), df


try:
    for index, row in union_dataset.iterrows():
        image_base64, df = find_image_from_name_dog(row['BreedName'])
        dataset_breed_dogs = df
        if row["Image"] and row["Type"] == "Dog" and not is_base64(row["Image"]):
            union_dataset.loc[index, 'Image'] = image_base64
except Exception as error:
    print(error)

union_dataset.to_csv("../data/dataset_union.csv")

<h3>Tratar valores nulos nos dados de gatos utilizando média de outro dataset</h3>

In [15]:
dataset_names_cat = pd.read_csv('../data/2022-licenced-cat-names.csv')

In [16]:
nomesGato = np.array(dataset_names_cat['names'])

def getNameCat():
    return np.random.choice(nomesGato)

union_dataset['AnimalName'] = union_dataset['AnimalName'].map(lambda name: getNameCat() if not name else name)

In [17]:
mesGato = np.array(union_dataset['AnimalBirthMonth'])

mesGatoFiltrado = list(filter(lambda mes: mes != None, mesGato))

def getBirthMonth():
    return np.random.choice(mesGatoFiltrado)

union_dataset['AnimalBirthMonth'] = union_dataset['AnimalBirthMonth'].map(lambda month: getBirthMonth() if not month else month)

In [ ]:
union_dataset.to_csv("../data/dataset_union.csv")

In [ ]:
union_dataset.insert(0, 'ID', range(len(union_dataset), 0, -1))